In [1]:
#Dependencies
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession
from lxml import etree
from requests_html import HTML



In [2]:
#utilts
import os
import json

def printSoupToHtml(soup, filename='soup.html'):
    # Creating a file if not exist
    if not os.path.exists(filename):
        with open(filename, 'w', encoding='utf-8') as file:
            file.write('')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(str(soup))
    return True

def printSoupToJson(soup, filename='soup.json'):
    # Creating a file if not exist
    if not os.path.exists(filename):
        with open(filename, 'w', encoding='utf-8') as file:
            file.write('')
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(json.dumps(soup))
    return True

In [3]:
import  json5 ,json

def getStringBetweenTwoWords(string, first, last):
    try:
        start = string.index(first) + len(first)
        end = string.index(last, start)
        return string[start:end]
    except ValueError:
        return ""
def convertStringToJson(string):
    x = json.dumps(string)
    printSoupToJson(x)

def extract_and_stringify_object(original):
    for i in range(0, len(original)):
        if original[i] == '{':
            start = i
            break
    for i in range(len(original) - 1, -1, -1):
        if original[i] == '}':
            end = i
            break  
    if end < len(original) - 1 and original[end + 1] != ',':
        original = original[:end + 1] + ',' + original[end + 1:]
    return json.dumps(json5.loads(original[start:end + 1]), indent=4)

def getJsonBetweenTwoWords(string, first, last):
    try:
        start = string.index(first) + len(first)
        end = string.index(last, start)
        return extract_and_stringify_object(string[start:end])
    except ValueError:
        return ""
async def scrapeJarir(url):  
    session = AsyncHTMLSession()
    response = await session.get(url)
    printSoupToHtml(response.content.decode(),"jarir.html")
    script_target_object=getStringBetweenTwoWords(response.content.decode(), 'window.__INITIAL_STATE__= ','</script>')
    strat_index=response.content.decode().index('window.__INITIAL_STATE__')
    end_index=response.content.decode().index('</script>',strat_index)
    script_target_object=response.content.decode()[strat_index:end_index].replace('window.__INITIAL_STATE__=','')
    printSoupToHtml(script_target_object)
    price=getStringBetweenTwoWords(script_target_object, 'final_price_ex_tax:',',')
    Title=getStringBetweenTwoWords(script_target_object, 'GTM_name:',',')
    images_string = getStringBetweenTwoWords(script_target_object, 'media_gallery','tsk').split("},{")
    #[{type:"image",image:"b/b/c/c/bbcc8c1c12a995f33ba9a9b8ab7c365703114777_631416.jpg",lab:"",pos:0},{type:"image",image:"f/4/f/e/f4fedbefb6cb954f43d7f6125bdbe259d7cf4d07_631416_1.jpg",lab:"",pos:1},{type:"image",image:"1/f/6/0/1f60183c0fc8f48082251e64b74fcc4dc7c74378_631416_2.jpg",lab:"",pos:2},{type:"image",image:"7/2/c/b/72cb5a45081b0696c661de124c2e0bb4758f2128_631416_3.jpg",lab:"",pos:3},{type:"image",image:"5/3/8/6/5386b16272b56e1c79cb085f7afc2167dacc6308_631416_4.jpg",lab:"",pos:4}],')
    images=[]
    for i in range(len(images_string)):
        images.append(getStringBetweenTwoWords(images_string[i], 'image:"','",lab'))
    return price , Title,images


In [4]:
#Test
url = 'https://www.jarir.com/sa-en/default-category/msi-clae-a1m-gaming-consoles-and-handheld-631416.html'
print( await scrapeJarir(url))

('2955.6522', '"MSI Claw A1M Handheld Gaming Console', ['b/b/c/c/bbcc8c1c12a995f33ba9a9b8ab7c365703114777_631416.jpg', 'f/4/f/e/f4fedbefb6cb954f43d7f6125bdbe259d7cf4d07_631416_1.jpg', '1/f/6/0/1f60183c0fc8f48082251e64b74fcc4dc7c74378_631416_2.jpg', '7/2/c/b/72cb5a45081b0696c661de124c2e0bb4758f2128_631416_3.jpg', '5/3/8/6/5386b16272b56e1c79cb085f7afc2167dacc6308_631416_4.jpg'])


In [5]:
# with open('jarir.html', 'r', encoding='utf-8') as file:
#     soup = BeautifulSoup(file, 'html.parser')
# printSoupToHtml(soup.prettify(), 'jarir_pretty.html')


In [6]:
import json


# with open('jarir.html', 'r', encoding='utf-8') as file:
# #     soup = file.read()
#     stringObj = getStringBetweenTwoWords(soup, 'original:', ',related').replace('!1', 'true').replace('!0', 'false')
session = AsyncHTMLSession()
response = await session.get('https://www.jarir.com/ae-ar/arabic-books-613657.html')
soup=response.content.decode()
#printSoupToHtml(soup,"jarir.html")
stringObj = getStringBetweenTwoWords(soup, 'original:', ',related').replace('!1', 'true').replace('!0', 'false')


obj = json.loads(extract_and_stringify_object(stringObj))
#printSoupToJson(obj, 'jarir.json')
# name_local = " ".join([
#     obj.get("arabicname_field1", ""),
#     obj.get("arabicname_field2", ""),
#     obj.get("arabicname_field3", ""),
#     obj.get("arabicname_field4", ""),
#     obj.get("arabicname_field5", ""),
#     obj.get("arabicname_field6", ""),
# ]).strip()

name_global = obj.get("name", "")
isAvailable = obj.get("check_availability_status", 0) == 1
PriceAfterDiscount = obj.get("final_price", obj.get("price", None))
description = obj.get("description", obj.get("short_description", ""))
rating = None  # Assuming rating information is not available in the provided JSON
images = [image["image"] for image in obj.get("media_gallery", []) if image.get("type") == "image"]
print(name_global, isAvailable, PriceAfterDiscount, description, rating, images)


‎انثروبولوجيا البطل في فلسفة التاريخ‎ True 11 default None ['8/6/6/6/866662d865334ab231a0156cca47189d8f30a889_613657.jpg', '6/2/9/7/62974938ff544b8c5cbaef5d0a07c3c0759bb253_613657_1.jpg']
